In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats

In [ ]:
nba_df = pd.read_csv("NBA_Data/NBA Player Stats(1950 - 2022).csv")
nba_df = nba_df[["Season", "Player", "G", "FG%", "3P%", "2P%", "MP", "PTS", "ORB", "DRB", "TRB", "AST", "STL", "BLK", "TOV"]]
games_1985_to_2000 = pd.read_csv("NBA_Data/games_data_1985_to_2000.csv")
games_2001_to_today = pd.read_csv("NBA_Data/games_data_2001_to_today.csv")
games_1985_to_2000 = games_1985_to_2000[["WL", "MIN", "FGM", "FGA", "FTM", "FTA", "OREB", "DREB", "AST", "STL", "BLK", "PTS"]]
games_2001_to_today = games_2001_to_today[["WL", "MIN", "FGM", "FGA", "FTM", "FTA", "OREB", "DREB", "AST", "STL", "BLK", "PTS"]]
finding_metric = pd.concat([games_1985_to_2000, games_2001_to_today])
finding_metric = finding_metric.dropna(axis=0,  how="any")
finding_metric.loc[finding_metric["WL"] == "W", "WL"] = 1
finding_metric.loc[finding_metric["WL"] == "L", "WL"] = 0

In [ ]:
finding_metric

In [ ]:
results = {}
for col in finding_metric.columns:
    if col == "WL":
        continue
    results[col] = stats.pearsonr(finding_metric[col], finding_metric["WL"])

In [ ]:
results

In [ ]:
outer_dict = {}
for col in finding_metric.columns:
    if col == "WL":
        continue
    inner_dict = {}
    max = int(finding_metric[col].max())
    min = int(finding_metric[col].min())
    # offset = range / 20
    for i in range(min, max):
        j = i+1
        temp = finding_metric.loc[(finding_metric[col] >= i) & (finding_metric[col] < j)]["WL"].mean()
        if not pd.isna(temp):
            if not (temp == 1 or temp == 0):
                inner_dict[i] = temp
    outer_dict[col] = inner_dict

In [ ]:
results = {}
for col in outer_dict:
    keys = []
    values = []
    for key, value in outer_dict[col].items():
        keys.append(key)
        values.append(value)
    results[col] = stats.pearsonr(keys, values)
results

In [ ]:
pvalue_cutoff = 0.05
statistic_cutoff = 0.5
important_results = {col:results[col] for col in results if (results[col].pvalue < pvalue_cutoff and np.abs(results[col].statistic) > statistic_cutoff)}
important_results

In [ ]:
finding_metric["PTS"].max()

In [ ]:
all_players_df = nba_df.loc[nba_df["Season"] > 1984]
all_players_df.head()

In [ ]:
year_1985_df = all_players_df.loc[(all_players_df["Season"] == 1985) & (all_players_df["PTS"] > 0)]
year_1985_df = year_1985_df.sort_values("PTS", ascending=False)

avg_pts_1985 = year_1985_df["PTS"].mean()
year_1985_df.head()
# percent_above = year_1985_df.loc[year_1985_df[]]

In [ ]:
specific_player_df = nba_df.loc[(nba_df["Player"] == "LeBron James") | (nba_df["Player"] == "Michael Jordan")]
specific_player_df

In [ ]:
#Lifetime Total Stats of each player (as of 2022 LeBron James/seasons)
total_games = specific_player_df["G"].groupby(specific_player_df["Player"]).sum()
total_mins = specific_player_df["MP"].groupby(specific_player_df["Player"]).sum()
total_pts = specific_player_df["PTS"].groupby(specific_player_df["Player"]).sum()
total_reb = specific_player_df["TRB"].groupby(specific_player_df["Player"]).sum()
total_ast = specific_player_df["AST"].groupby(specific_player_df["Player"]).sum()
total_stl = specific_player_df["STL"].groupby(specific_player_df["Player"]).sum()
total_blk = specific_player_df["BLK"].groupby(specific_player_df["Player"]).sum()
total_tov = specific_player_df["TOV"].groupby(specific_player_df["Player"]).sum()
total_summary_df = pd.DataFrame({"Total Games Played": total_games, "Total Mins Played": total_mins, "Total Points Scored": total_pts,
                           "Total Rebounds": total_reb, "Total Assists": total_ast, "Total Steals": total_stl, "Total Blocks": total_blk, 
                           "Total Turnovers": total_tov})
total_summary_df

In [ ]:
#Lifetime Average Stats of each player (as of 2022 LeBron James/seasons)
mean_games = specific_player_df["G"].groupby(specific_player_df["Player"]).mean()
mean_fgp = specific_player_df["FG%"].groupby(specific_player_df["Player"]).mean()
mean_3pc = specific_player_df["3P%"].groupby(specific_player_df["Player"]).mean()
mean_2pc = specific_player_df["2P%"].groupby(specific_player_df["Player"]).mean()
mean_mins = specific_player_df["MP"].groupby(specific_player_df["Player"]).mean()
mean_pts = specific_player_df["PTS"].groupby(specific_player_df["Player"]).mean()
mean_reb = specific_player_df["TRB"].groupby(specific_player_df["Player"]).mean()
mean_ast = specific_player_df["AST"].groupby(specific_player_df["Player"]).mean()
mean_stl = specific_player_df["STL"].groupby(specific_player_df["Player"]).mean()
mean_blk = specific_player_df["BLK"].groupby(specific_player_df["Player"]).mean()
mean_tov = specific_player_df["TOV"].groupby(specific_player_df["Player"]).mean()
mean_summary_df = pd.DataFrame({"Avg. Games Played": mean_games, "Avg. Mins Played": mean_mins, "Avg. Points Scored": mean_pts,
                                "Avg. Field Goal %": mean_fgp, "Avg. 3 Point %": mean_3pc, "Avg. 2 Point %": mean_2pc,
                           "Avg. Rebounds": mean_reb, "Avg. Assists": mean_ast, "Avg. Steals": mean_stl, "Avg. Blocks": mean_blk, 
                           "Avg. Turnovers": mean_tov})
mean_summary_df

In [ ]:
mj_specific_player_df = specific_player_df.loc[(specific_player_df["Player"] == "Michael Jordan")]
mj_efficiency = mj_specific_player_df["PTS"] / mj_specific_player_df["MP"]
mj_efficiency = mj_efficiency.reset_index(drop = True)
mj_efficiency

In [ ]:
lj_specific_player_df = specific_player_df.loc[(specific_player_df["Player"] == "LeBron James")]
lj_efficiency = lj_specific_player_df["PTS"] / lj_specific_player_df["MP"]
lj_efficiency = lj_efficiency.reset_index(drop = True)
lj_efficiency

In [ ]:
mj_plot, = plt.plot(range(len(mj_efficiency)), mj_efficiency, color="red", label="Michael Jordan")
lj_plot, = plt.plot(range(len(lj_efficiency)), lj_efficiency, color="purple", label="LeBron James")
plt.xticks(range(len(lj_efficiency)))
plt.xlabel("Per Season")
plt.ylabel("Points Per Minute Played")
plt.legend(handles=[mj_plot, lj_plot], loc="best")
plt.savefig("Charts/efficiency_pts_min.png")
plt.show()

In [ ]:
each_game_stats = pd.read_csv("NBA_Data/MJ_LJ_Games.csv")
each_game_stats.head()  

In [ ]:
each_game_stats.rename(columns={"PLAYER_NAME": "Player", "FG_PCT": "FG%", "FG3_PCT": "3P%"}, inplace=True)
each_game_stats

In [ ]:
each_game_stats = each_game_stats[["Season", "Player", "MATCHUP", "WL", "PTS", "FG%", "OREB", "DREB", "AST", "STL", "BLK", "TOV"]]
each_game_stats.head()

In [ ]:
#Check for na values
each_game_stats.count()

In [ ]:
each_game_stats.dropna(inplace=True)

In [ ]:
each_game_stats.count()

In [ ]:
#Calculating linear regression for MJ, points per game
# Filter the DataFrame for "Michael Jordan"
michael_jordan_stats = each_game_stats[each_game_stats['Player'] == 'Michael Jordan']

# Create a scatter plot for the "PTS" column
plt.figure(figsize=(10, 6))
plt.scatter(michael_jordan_stats.index, michael_jordan_stats['PTS'], color='orange', label='PTS')
plt.title('Michael Jordan - Points per Game')
plt.xlabel('Game Number')
plt.ylabel('Points')
plt.legend()
plt.grid(True)

# Calculate the linear regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(michael_jordan_stats.index, michael_jordan_stats['PTS'])

# Calculate the regression line for the entire x-axis range
x_range = range(1, 1501)  # Define the full x-axis range
regression_line = slope * x_range + intercept


# Plot the linear regression line for the full x-axis range
plt.plot(x_range, regression_line, color='black', label='Linear Regression')

# Plot the linear regression line
plt.legend()
plt.savefig('Charts/mj_pts_regression.png')
plt.show()

In [ ]:
#Calculating linear regression for LJ, points per game
# Filter the DataFrame for "Michael Jordan"
lebron_james_stats = each_game_stats[each_game_stats['Player'] == 'LeBron James']

# Create a scatter plot for the "PTS" column
plt.figure(figsize=(10, 6))
plt.scatter(lebron_james_stats.index, lebron_james_stats['PTS'], color='#CBC3E3', label='PTS')
plt.title('LeBron James - Points per Game')
plt.xlabel('Game Number')
plt.ylabel('Points')
plt.legend()
plt.grid(True)

# Calculate the linear regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(lebron_james_stats.index, lebron_james_stats['PTS'])

# Print the slope of the linear regression line
print("Linear Regression Slope:", slope)

regression_line = slope * lebron_james_stats.index + intercept

# Plot the linear regression line
plt.plot(lebron_james_stats.index, regression_line, color='black', label='Linear Regression')
plt.savefig('Charts/lj_pts_regression.png')
plt.legend()
plt.show()

In [ ]:
#Lifetime Points Stats (Offensive/each game)
mean_pts = each_game_stats["PTS"].groupby(each_game_stats["Player"]).mean()
median_pts = each_game_stats["PTS"].groupby(each_game_stats["Player"]).median()
var_pts = each_game_stats["PTS"].groupby(each_game_stats["Player"]).var()
std_dev_pts = each_game_stats["PTS"].groupby(each_game_stats["Player"]).std()
std_err_pts = each_game_stats["PTS"].groupby(each_game_stats["Player"]).sem()

summary_lifetime_pts_df = pd.DataFrame({"Mean Points Scored": mean_pts, "Median Points Scored": median_pts, "Variance of Points Scored": var_pts,
                               "Std. Dev of Points Scored": std_dev_pts, "Std. Error of Points Scored": std_err_pts})
summary_lifetime_pts_df

In [ ]:
#Lifetime Field Goal % (Offensive/each game)
mean_fgp = each_game_stats["FG%"].groupby(each_game_stats["Player"]).mean()
median_fgp = each_game_stats["FG%"].groupby(each_game_stats["Player"]).median()
var_fgp = each_game_stats["FG%"].groupby(each_game_stats["Player"]).var()
std_dev_fgp = each_game_stats["FG%"].groupby(each_game_stats["Player"]).std()
std_err_fgp = each_game_stats["FG%"].groupby(each_game_stats["Player"]).sem()

summary_lifetime_fgp_df = pd.DataFrame({"Mean Field Goal %": mean_fgp, "Median Field Goal %": median_fgp, "Variance of Field Goal %": var_fgp,
                               "Std. Dev of Field Goal %": std_dev_fgp, "Std. Error of Field Goal %": std_err_fgp})
summary_lifetime_fgp_df

In [ ]:
#Lifetime Offensive Rebounds (Offensive/each game)
mean_oreb = each_game_stats["OREB"].groupby(each_game_stats["Player"]).mean()
median_oreb = each_game_stats["OREB"].groupby(each_game_stats["Player"]).median()
var_oreb = each_game_stats["OREB"].groupby(each_game_stats["Player"]).var()
std_dev_oreb = each_game_stats["OREB"].groupby(each_game_stats["Player"]).std()
std_err_oreb = each_game_stats["OREB"].groupby(each_game_stats["Player"]).sem()

summary_lifetime_oreb_df = pd.DataFrame({"Mean Offensive Rebounds": mean_oreb, "Median Offensive Rebounds": median_oreb,
                                     "Variance of Offensive Rebounds": var_oreb, "Std. Dev of Offensive Rebounds": std_dev_oreb,
                                     "Std. Error of Offensive Rebounds": std_err_oreb})
summary_lifetime_oreb_df

In [ ]:
#Lifetime Assists (Offensive/each game)
mean_ast = each_game_stats["AST"].groupby(each_game_stats["Player"]).mean()
median_ast = each_game_stats["AST"].groupby(each_game_stats["Player"]).median()
var_ast = each_game_stats["AST"].groupby(each_game_stats["Player"]).var()
std_dev_ast = each_game_stats["AST"].groupby(each_game_stats["Player"]).std()
std_err_ast = each_game_stats["AST"].groupby(each_game_stats["Player"]).sem()

summary_lifetime_ast_df = pd.DataFrame({"Mean Assists": mean_ast, "Median Assists": median_ast,
                                     "Variance of Assists": var_ast, "Std. Dev of Assists": std_dev_ast,
                                     "Std. Error of Assists": std_err_ast})
summary_lifetime_ast_df

In [ ]:
combined_offensive1 = pd.merge(summary_lifetime_pts_df, summary_lifetime_ast_df, on = "Player")
combined_offensive1 = pd.merge(combined_offensive1, summary_lifetime_oreb_df, on = "Player")
mean_values_chart1 = combined_offensive1[["Mean Points Scored", "Mean Assists", "Mean Offensive Rebounds"]]
mean_values_chart1.plot(kind="bar")
plt.xticks(rotation = 45)
plt.legend(loc="upper center", bbox_to_anchor=(0.4, 1.015))
plt.savefig("Charts/offensive1.png")
plt.show()

In [ ]:
mean_values_chart2 = summary_lifetime_fgp_df["Mean Field Goal %"]
mean_values_chart2.plot(kind="bar")
plt.xticks(rotation = 45)
plt.legend(loc="upper center", bbox_to_anchor=(0.50, 1))
plt.savefig("Charts/offensive2.png")
plt.show()

In [ ]:
#Lifetime Defensive Rebounds (Defensive/each game)
mean_dreb = each_game_stats["DREB"].groupby(each_game_stats["Player"]).mean()
median_dreb = each_game_stats["DREB"].groupby(each_game_stats["Player"]).median()
var_dreb = each_game_stats["DREB"].groupby(each_game_stats["Player"]).var()
std_dev_dreb = each_game_stats["DREB"].groupby(each_game_stats["Player"]).std()
std_err_dreb = each_game_stats["DREB"].groupby(each_game_stats["Player"]).sem()

summary_lifetime_dreb_df = pd.DataFrame({"Mean Defensive Rebounds": mean_dreb, "Median Defensive Rebounds": median_dreb,
                                     "Variance of Defensive Rebounds": var_dreb, "Std. Dev of Defensive Rebounds": std_dev_dreb,
                                     "Std. Error of Defensive Rebounds": std_err_dreb})
summary_lifetime_dreb_df

In [ ]:
#Lifetime Steals (Defensive/each game)
mean_stl = each_game_stats["STL"].groupby(each_game_stats["Player"]).mean()
median_stl = each_game_stats["STL"].groupby(each_game_stats["Player"]).median()
var_stl = each_game_stats["STL"].groupby(each_game_stats["Player"]).var()
std_dev_stl = each_game_stats["STL"].groupby(each_game_stats["Player"]).std()
std_err_stl = each_game_stats["STL"].groupby(each_game_stats["Player"]).sem()

summary_lifetime_stl_df = pd.DataFrame({"Mean Steals": mean_stl, "Median Steals": median_stl,
                                     "Variance of Steals": var_stl, "Std. Dev of Steals": std_dev_stl,
                                     "Std. Error of Steals": std_err_stl})
summary_lifetime_stl_df

In [ ]:
#Lifetime Blocks (Defensive/each game)
mean_blk = each_game_stats["BLK"].groupby(each_game_stats["Player"]).mean()
median_blk = each_game_stats["BLK"].groupby(each_game_stats["Player"]).median()
var_blk = each_game_stats["BLK"].groupby(each_game_stats["Player"]).var()
std_dev_blk = each_game_stats["BLK"].groupby(each_game_stats["Player"]).std()
std_err_blk = each_game_stats["BLK"].groupby(each_game_stats["Player"]).sem()

summary_lifetime_blk_df = pd.DataFrame({"Mean Blocks": mean_blk, "Median Blocks": median_blk,
                                     "Variance of Blocks": var_blk, "Std. Dev of Blocks": std_dev_blk,
                                     "Std. Error of Blocks": std_err_blk})
summary_lifetime_blk_df

In [ ]:
combined_def_stats = pd.merge(summary_lifetime_dreb_df, summary_lifetime_stl_df, on="Player")
combined_def_stats = pd.merge(combined_def_stats, summary_lifetime_blk_df, on="Player")
mean_def_chart = combined_def_stats[["Mean Defensive Rebounds", "Mean Steals", "Mean Blocks"]]
mean_def_chart.plot.barh()
plt.legend(loc="upper left", bbox_to_anchor=(0.50, 1))
plt.savefig("Charts/defensive.png")
plt.show()

In [ ]:
#Lifetime Turnover (misc/each game)
mean_tov = each_game_stats["TOV"].groupby(each_game_stats["Player"]).mean()
median_tov = each_game_stats["TOV"].groupby(each_game_stats["Player"]).median()
var_tov = each_game_stats["TOV"].groupby(each_game_stats["Player"]).var()
std_dev_tov = each_game_stats["TOV"].groupby(each_game_stats["Player"]).std()
std_err_tov = each_game_stats["TOV"].groupby(each_game_stats["Player"]).sem()

summary_lifetime_tov_df = pd.DataFrame({"Mean Turnovers": mean_tov, "Median Turnovers": median_tov,
                                     "Variance of Turnovers": var_tov, "Std. Dev of Turnovers": std_dev_tov,
                                     "Std. Error of Turnovers": std_err_tov})
summary_lifetime_tov_df

In [ ]:
mj_stats = each_game_stats.loc[each_game_stats["Season"] == 1991]
mj_stats

In [ ]:
#PTS stats when MJ won championship 1991 (age 27) **offensive stats
mean_pts_mj = mj_stats["PTS"].groupby(mj_stats["Player"]).mean() 
median_pts_mj = mj_stats["PTS"].groupby(mj_stats["Player"]).median()
var_pts_mj = mj_stats["PTS"].groupby(mj_stats["Player"]).var()
std_dev_pts_mj = mj_stats["PTS"].groupby(mj_stats["Player"]).std()
std_err_pts_mj = mj_stats["PTS"].groupby(mj_stats["Player"]).sem()

summary_1991_pts_df = pd.DataFrame({"Mean Points Scored": mean_pts_mj, "Median Points Scored": median_pts_mj, "Variance of Points Scored": var_pts_mj,
                               "Std. Dev of Points Scored": std_dev_pts_mj, "Std. Error of Points Scored": std_err_pts_mj})
summary_1991_pts_df

In [ ]:
#FG% stats when MJ won championship 1991 (age 27) **offensive stats
mean_fgp_mj = mj_stats["FG%"].groupby(mj_stats["Player"]).mean()
median_fgp_mj = mj_stats["FG%"].groupby(mj_stats["Player"]).median()
var_fgp_mj = mj_stats["FG%"].groupby(mj_stats["Player"]).var()
std_dev_fgp_mj = mj_stats["FG%"].groupby(mj_stats["Player"]).std()
std_err_fgp_mj = mj_stats["FG%"].groupby(mj_stats["Player"]).sem()

summary_1991_fgp_df = pd.DataFrame({"Mean Field Goal %": mean_fgp_mj, "Median Field Goal %": median_fgp_mj, "Variance of Field Goal %": var_fgp_mj,
                               "Std. Dev of Field Goal %": std_dev_fgp_mj, "Std. Error of Field Goal %": std_err_fgp_mj})
summary_1991_fgp_df

In [ ]:
#OREB stats when MJ won championship 1991 (age 27) **offensive stats
mean_oreb_mj = mj_stats["OREB"].groupby(mj_stats["Player"]).mean()
median_oreb_mj = mj_stats["OREB"].groupby(mj_stats["Player"]).median()
var_oreb_mj = mj_stats["OREB"].groupby(mj_stats["Player"]).var()
std_dev_oreb_mj = mj_stats["OREB"].groupby(mj_stats["Player"]).std()
std_err_oreb_mj = mj_stats["OREB"].groupby(mj_stats["Player"]).sem()

summary_1991_oreb_df = pd.DataFrame({"Mean Offensive Rebounds": mean_oreb_mj, "Median Offensive Rebounds": median_oreb_mj,
                                     "Variance of Offensive Rebounds": var_oreb_mj, "Std. Dev of Offensive Rebounds": std_dev_oreb_mj,
                                     "Std. Error of Offensive Rebounds": std_err_oreb_mj})
summary_1991_oreb_df

In [ ]:
#AST stats when MJ won championship 1991 (age 27) **offensive stats
mean_ast_mj = mj_stats["AST"].groupby(mj_stats["Player"]).mean()
median_ast_mj = mj_stats["AST"].groupby(mj_stats["Player"]).median()
var_ast_mj = mj_stats["AST"].groupby(mj_stats["Player"]).var()
std_dev_ast_mj = mj_stats["AST"].groupby(mj_stats["Player"]).std()
std_err_ast_mj = mj_stats["AST"].groupby(mj_stats["Player"]).sem()

summary_1991_ast_df = pd.DataFrame({"Mean Assists": mean_oreb_mj, "Median Assists": median_oreb_mj,
                                     "Variance of Assists": var_oreb_mj, "Std. Dev of Assists": std_dev_oreb_mj,
                                     "Std. Error of Assists": std_err_oreb_mj})
summary_1991_ast_df

In [ ]:
#TOV stats when MJ won championship 1991 (age 27) **offensive stats
mean_tov_mj = mj_stats["TOV"].groupby(mj_stats["Player"]).mean()
median_tov_mj = mj_stats["TOV"].groupby(mj_stats["Player"]).median()
var_tov_mj = mj_stats["TOV"].groupby(mj_stats["Player"]).var()
std_dev_tov_mj = mj_stats["TOV"].groupby(mj_stats["Player"]).std()
std_err_tov_mj = mj_stats["TOV"].groupby(mj_stats["Player"]).sem()

summary_1991_tov_df = pd.DataFrame({"Mean Turnovers": mean_tov_mj, "Median Turnovers": median_tov_mj,
                                     "Variance of Turnovers": var_tov_mj, "Std. Dev of Turnovers": std_dev_tov_mj,
                                     "Std. Error of Turnovers": std_err_tov_mj})
summary_1991_tov_df

In [ ]:
lj_stats = each_game_stats.loc[each_game_stats["Season"] == 2012]
lj_stats

In [ ]:
#PTS stats when LJ won championship 2012 (age 28) **offensive stats
mean_pts_lj = lj_stats["PTS"].groupby(lj_stats["Player"]).mean()
median_pts_lj = lj_stats["PTS"].groupby(lj_stats["Player"]).median()
var_pts_lj = lj_stats["PTS"].groupby(lj_stats["Player"]).var()
std_dev_pts_lj = lj_stats["PTS"].groupby(lj_stats["Player"]).std()
std_err_pts_lj = lj_stats["PTS"].groupby(lj_stats["Player"]).sem()

summary_2012_pts_df = pd.DataFrame({"Mean Points Scored": mean_pts_lj, "Median Points Scored": median_pts_lj, "Variance of Points Scored": var_pts_lj,
                               "Std. Dev of Points Scored": std_dev_pts_lj, "Std. Error of Points Scored": std_err_pts_lj})
summary_2012_pts_df

In [ ]:
#FG% stats when LJ won championship 2012 (age 28) **offensive stats
mean_fgp_lj = lj_stats["FG%"].groupby(lj_stats["Player"]).mean()
median_fgp_lj = lj_stats["FG%"].groupby(lj_stats["Player"]).median()
var_fgp_lj = lj_stats["FG%"].groupby(lj_stats["Player"]).var()
std_dev_fgp_lj = lj_stats["FG%"].groupby(lj_stats["Player"]).std()
std_err_fgp_lj = lj_stats["FG%"].groupby(lj_stats["Player"]).sem()

summary_2012_fgp_df = pd.DataFrame({"Mean Field Goal %": mean_fgp_lj, "Median Field Goal %": median_fgp_lj, "Variance of Field Goal %": var_fgp_lj,
                               "Std. Dev of Field Goal %": std_dev_fgp_lj, "Std. Error of Field Goal %": std_err_fgp_lj})
summary_2012_fgp_df

In [ ]:
#OREB stats when LJ won championship 2012 (age 28) **offensive stats
mean_oreb_lj = lj_stats["OREB"].groupby(lj_stats["Player"]).mean()
median_oreb_lj = lj_stats["OREB"].groupby(lj_stats["Player"]).median()
var_oreb_lj = lj_stats["OREB"].groupby(lj_stats["Player"]).var()
std_dev_oreb_lj = lj_stats["OREB"].groupby(lj_stats["Player"]).std()
std_err_oreb_lj = lj_stats["OREB"].groupby(lj_stats["Player"]).sem()

summary_2012_oreb_df = pd.DataFrame({"Mean Offensive Rebounds": mean_oreb_lj, "Median Offensive Rebounds": median_oreb_lj,
                                     "Variance of Offensive Rebounds": var_oreb_lj, "Std. Dev of Offensive Rebounds": std_dev_oreb_lj,
                                     "Std. Error of Offensive Rebounds": std_err_oreb_lj})
summary_2012_oreb_df

In [ ]:
#AST stats when LJ won championship 2012 (age 28) **offensive stats
mean_ast_lj = lj_stats["AST"].groupby(lj_stats["Player"]).mean()
median_ast_lj = lj_stats["AST"].groupby(lj_stats["Player"]).median()
var_ast_lj = lj_stats["AST"].groupby(lj_stats["Player"]).var()
std_dev_ast_lj = lj_stats["AST"].groupby(lj_stats["Player"]).std()
std_err_ast_lj = lj_stats["AST"].groupby(lj_stats["Player"]).sem()

summary_2012_ast_df = pd.DataFrame({"Mean Assists": mean_ast_lj, "Median Assists": median_ast_lj,
                                     "Variance of Assists": var_ast_lj, "Std. Dev of Assists": std_dev_ast_lj,
                                     "Std. Error of Assists": std_err_ast_lj})
summary_2012_ast_df

In [ ]:
#TOV stats when LJ won championship 2012 (age 28) **offensive stats
mean_tov_lj = lj_stats["TOV"].groupby(lj_stats["Player"]).mean()
median_tov_lj = lj_stats["TOV"].groupby(lj_stats["Player"]).median()
var_tov_lj = lj_stats["TOV"].groupby(lj_stats["Player"]).var()
std_dev_tov_lj = lj_stats["TOV"].groupby(lj_stats["Player"]).std()
std_err_tov_lj = lj_stats["TOV"].groupby(lj_stats["Player"]).sem()

summary_2012_tov_df = pd.DataFrame({"Mean Turnovers": mean_ast_lj, "Median Turnovers": median_ast_lj,
                                     "Variance of Turnovers": var_ast_lj, "Std. Dev of Turnovers": std_dev_ast_lj,
                                     "Std. Error of Turnovers": std_err_ast_lj})
summary_2012_tov_df

In [ ]:
champ_combined = pd.merge(summary_1991_pts_df, summary_2012_pts_df, how="outer")
champ_combined

In [ ]:
combined_best_season = specific_player_df.loc[(specific_player_df["Season"] == 1988) | (specific_player_df["Season"] == 2010)]
combined_best_season = combined_best_season.set_index("Player")
combined_best_season

In [ ]:
combined_best_plot = combined_best_season[["DRB", "STL", "BLK"]]
combined_best_plot["DRB"] = combined_best_season["DRB"] / combined_best_season["MP"]
combined_best_plot["STL"] = combined_best_season["STL"] / combined_best_season["MP"]
combined_best_plot["BLK"] = combined_best_season["BLK"] / combined_best_season["MP"]
combined_best_plot.plot.barh()
plt.legend(loc="upper left", bbox_to_anchor=(0.50, 1))
plt.title("Best Season Defensive Stats (per minute played)")
plt.savefig("Charts/defensive_best_season.png")
plt.show()

In [ ]:
fgp_best_season = combined_best_season["FG%"]
fgp_best_season.plot(kind="bar")
plt.xticks(rotation = 45)
plt.legend(loc="upper center", bbox_to_anchor=(0.50, 1))
plt.savefig("Charts/fgp_best_season.png")
plt.show()

In [ ]:
offensive_best_season = combined_best_season[["PTS", "AST", "ORB"]]
offensive_best_season["PTS"] = combined_best_season["PTS"] / combined_best_season["MP"]
offensive_best_season["AST"] = combined_best_season["AST"] / combined_best_season["MP"]
offensive_best_season["ORB"] = combined_best_season["ORB"] / combined_best_season["MP"]
offensive_best_season = offensive_best_season.rename(columns={"PTS": "Points", "AST": "Assists", "ORB": "Offensive Rebounds"})
offensive_best_season.plot(kind="bar")
plt.xticks(rotation = 45)
plt.legend(loc="upper center", bbox_to_anchor=(0.4, 1.015))
plt.title("Best Season Offensive Stats (per minute played)")
plt.savefig("Charts/offensive_best_season.png")
plt.show()